# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
cities_filepath = '../output_data/cities.csv'
cities_data = pd.read_csv(cities_filepath) 

cities_data.head()


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [11]:
#Store Latitude and Longitude into locations
locations = cities_data[['Lat', 'Lng']].astype(float)

#Store Humidity to be used as the Weight
humidity = cities_data['Humidity'].astype(float)

In [13]:
#Create Heatmap

figure_layout = {'width' : '1000px',
                'height' : '500px',
                'border': '1px solid black',
                'padding': '1px',
                 'margin': '0 auto 0 auto'
                }

fig = gmaps.figure(layout = figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
#Narrow down the DataFrame to my ideal Weather Criteria 

fav_weather_df = cities_data.loc[cities_data['Max Temp']<75]
fav_weather_df = fav_weather_df.loc[fav_weather_df['Max Temp']>65]
fav_weather_df = fav_weather_df.loc[fav_weather_df['Wind Speed']<10]
fav_weather_df = fav_weather_df.loc[fav_weather_df['Cloudiness']<25]
fav_weather_df = fav_weather_df.loc[fav_weather_df['Humidity']<50]
fav_weather_df = fav_weather_df.loc[fav_weather_df['Humidity']>30]

fav_weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
338,338,port shepstone,2,ZA,1558378854,47,-30.74,30.45,69.54,5.32
346,346,chiredzi,0,ZW,1558378856,42,-21.05,31.67,65.76,0.78
360,360,fort nelson,20,CA,1558378858,42,58.81,-122.69,68.00,5.82
533,533,hakkari,0,FI,1558378896,46,61.34,23.75,69.80,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
#Create hotel_df based on my narrowed down list of cities. Limit Columns to City, Country, Latitude, & Longitude.
hotel_df = pd.DataFrame(fav_weather_df, columns = ['City', 'Country', 'Lat', 'Lng'])

#Add "Hotel Name" column.
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Hotel Name
338,port shepstone,ZA,-30.74,30.45,
346,chiredzi,ZW,-21.05,31.67,
360,fort nelson,CA,58.81,-122.69,
533,hakkari,FI,61.34,23.75,


In [16]:
#Create an empty place to store Hotel Info from API call
hotels = []

#Iterate through dataframe to find nearest Hotel
for index, row in hotel_df.iterrows():
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    latitude = hotel_df.loc[index]['Lat']
    longitude = hotel_df.loc[index]['Lng']
    params = {'location' : f'{latitude},{longitude}',
             'radius' : 5000,
             'types' : 'hotel',
             'key': g_key}
    response = requests.get(base_url, params=params).json()
    results = response['results']
    try:
        hotels.append(results[0]['name'])
    except:
        hotels.appened('')

#Update dataframe with Hotel Info from API 
hotel_df['Hotel Name'] = hotels
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
338,port shepstone,ZA,-30.74,30.45,Port Shepstone
346,chiredzi,ZW,-21.05,31.67,Chiredzi
360,fort nelson,CA,58.81,-122.69,Fort Nelson
533,hakkari,FI,61.34,23.75,Lempäälä


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
# Display figure

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…